# 第九章 评估（上）——存在一个简单的正确答案时

 - [一、环境配置](#一、环境配置)
     - [1.1 加载 API 密钥和一些 Python 库。](#1.1-加载-API-密钥和一些-Python-库。)
     - [1.2 获取相关产品和类别](#1.2-获取相关产品和类别)
 - [二、找出相关产品和类别名称](#二、找出相关产品和类别名称)
 - [三、在一些查询上进行评估](#三、在一些查询上进行评估)
 - [四、更难的测试用例](#四、更难的测试用例)
 - [五、修改指令以处理难测试用例](#五、修改指令以处理难测试用例)
 - [六、在难测试用例上评估修改后的指令](#六、在难测试用例上评估修改后的指令)
 - [七、回归测试：验证模型在以前的测试用例上仍然有效](#七、回归测试：验证模型在以前的测试用例上仍然有效)
 - [八、收集开发集进行自动化测试](#八、收集开发集进行自动化测试)
 - [九、通过与理想答案比较来评估测试用例](#九、通过与理想答案比较来评估测试用例)
 - [十、在所有测试用例上运行评估，并计算正确的用例比例](#十、在所有测试用例上运行评估，并计算正确的用例比例)


在之前的章节中，我们展示了如何使用 LLM 构建应用程序，包括评估输入、处理输入以及在向用户显示输出之前进行最终输出检查。

构建这样的系统后，如何知道它的工作情况？甚至在部署后并让用户使用它时，如何跟踪它的运行情况，发现任何缺陷，并持续改进系统的答案质量？

在本章中，我们想与您分享一些最佳实践，用于<span style="color:red">评估 LLM 的输出</span>。

构建基于 LLM 的应用程序与传统的监督学习应用程序有所不同。由于可以快速构建基于 LLM 的应用程序，因此评估方法通常不从测试集开始。相反，通常会逐渐建立一组测试示例。

在传统的监督学习环境中，需要收集训练集、开发集或保留交叉验证集，然后在整个开发过程中使用它们。

然而，如果能够在几分钟内指定 Prompt，并在几个小时内得到相应结果，那么暂停很长时间去收集一千个测试样本将是一件极其痛苦的事情。因为现在，可以在的情况下获得这个成果。

因此，在使用 LLM 构建应用程序时，您将体会到<span style="color:red">如下过程</span>：

首先，您会在只有一到三个样本的小样本中调整 Prompt，并尝试让 Prompt 在它们身上起作用。

然后，当系统进行进一步的测试时，您可能会遇到一些棘手的例子。Prompt 在它们身上不起作用，或者算法在它们身上不起作用。

这就是使用 ChatGPT API 构建应用程序的开发者所经历的挑战。

在这种情况下，您可以将这些额外的几个示例添加到您正在测试的集合中，以机会主义地添加其他棘手的示例。

最终，您已经添加了足够的这些示例到您缓慢增长的开发集中，以至于通过手动运行每个示例来测试 Prompt 变得有些不方便。

然后，您开始开发在这些小示例集上用于衡量性能的指标，例如平均准确性。

这个过程的一个有趣方面是，如果您觉得您的系统已经足够好了，您可以随时停在那里，不再改进它。事实上，许多已部署的应用程序停在第一或第二个步骤，并且运行得非常好。

需要注意的是，有很多大模型的应用程序没有实质性的风险，即使它没有给出完全正确的答案。

但是，对于部分高风险应用，如果存在偏见或不适当的输出可能对某人造成伤害，那么收集测试集、严格评估系统的性能、确保在使用之前它能够做正确的事情，就变得更加重要。

但是，如果您只是使用它来总结文章供自己阅读，而不是给别人看，那么可能造成的危害风险更小，您可以在这个过程中早早停止，而不必去花费更大的代价去收集更大的数据集。

## 一、环境配置

### 1.1 加载 API 密钥和一些 Python 库。

同上一章，我们首先需要配置使用 OpenAI API 的环境

In [3]:
import os
import utils_chap8_ch

from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
client = OpenAI(api_key=openai_api_key)

def get_completion_from_messages(messages, 
                                 model ="gpt-4o-mini", 
                                 temperature=0, 
                                 max_tokens=1000):
    '''
    封装一个支持更多参数的自定义访问 gpt-4o-mini的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为gpt-4o-mini(ChatGPT)
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = client.chat.completions.create(
        model= model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message.content

### 1.2 获取相关产品和类别

我们要获取前几章中提到的产品目录中的产品和类别列表。

In [5]:
products_and_category = utils_chap8_ch.get_products_and_category()
products_and_category

{'电脑和笔记本': ['TechPro Ultrabook',
  'BlueWave Gaming Laptop',
  'PowerLite Convertible',
  'TechPro Desktop',
  'BlueWave Chromebook'],
 '智能手机和配件': ['SmartX ProPhone',
  'MobiTech PowerCase',
  'SmartX MiniPhone',
  'MobiTech Wireless Charger',
  'SmartX EarBuds'],
 '电视和家庭影院系统': ['CineView 4K TV',
  'SoundMax Home Theater',
  'CineView 8K TV',
  'SoundMax Soundbar',
  'CineView OLED TV'],
 '游戏机和配件': ['GameSphere X',
  'ProGamer Controller',
  'GameSphere Y',
  'ProGamer Racing Wheel',
  'GameSphere VR Headset'],
 '音频设备': ['AudioPhonic Noise-Canceling Headphones',
  'WaveSound Bluetooth Speaker',
  'AudioPhonic True Wireless Earbuds',
  'WaveSound Soundbar',
  'AudioPhonic Turntable'],
 '相机和摄像机': ['FotoSnap DSLR Camera',
  'ActionCam 4K',
  'FotoSnap Mirrorless Camera',
  'ZoomMaster Camcorder',
  'FotoSnap Instant Camera']}

## 二、找出相关产品和类别名称

In [14]:
def find_category_and_product_v1(user_input, products_and_category):
    """
    从用户输入中获取到产品和类别

    参数：
    user_input：用户的查询
    products_and_category：产品类型和对应产品的字典
    """
    
    delimiter = "####"

    system_message = f"""
    你将提供服务查询。
    服务查询将使用{delimiter}字符分隔。

    仅输出一个 Python 对象列表，其中每个对象具有以下格式：
        'category': <必须是在下面的允许产品列表中找到的类别>;
        'products': <必须是在下面的允许产品列表中找到的产品>;

    类别和产品必须在客户输入中找到。
    如果客户输入提及了类别，则将其与允许产品列表中的正确产品相关联。
    如果客户输入提及了产品，则将其与允许类别中的正确类别相关联。
    如果未找到符合要求的产品或类别，则输出空列表。

    根据产品名称和产品类别与客户服务查询的相关性，列出所有相关的产品。
    不要从产品的名称中假设任何特性或属性，如相对质量或价格。
    仅输出 Python 对象列表，不包含其他字符信息。

    允许的产品以JSON格式提供。
    每个项的键表示类别。
    每个项的值是该类别中的产品列表。
    允许的产品：{products_and_category}  
    """
    
    # 添加一个简单的few-shot示例
    few_shot_user_1 = """我想要最贵的电脑。"""
    few_shot_assistant_1 = """ 
    [{'category': '电脑和笔记本', \
    'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)

## 三、在一些查询上进行评估

In [15]:
# 第一个评估的查询
customer_msg_0 = f"""如果我预算有限，我可以买哪款电视？"""

products_by_category_0 = find_category_and_product_v1(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

    [{'category': '电视和家庭影院系统', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]


In [16]:
customer_msg_1 = f"""我需要一个智能手机的充电器"""

products_by_category_1 = find_category_and_product_v1(customer_msg_1,
                                                      products_and_category)
print(products_by_category_1)

[{'category': '智能手机和配件', 'products': ['MobiTech PowerCase', 'MobiTech Wireless Charger']}]


In [12]:
customer_msg_2 = f"""
你们有哪些电脑？"""

products_by_category_2 = find_category_and_product_v1(customer_msg_2,
                                                      products_and_category)
products_by_category_2

"[{'category': '电脑和笔记本', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]"

In [13]:
customer_msg_3 = f"""
告诉我关于smartx pro手机和fotosnap相机的信息，那款DSLR的。
另外，你们有哪些电视？"""

products_by_category_3 = find_category_and_product_v1(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

[
    {'category': '智能手机和配件', 'products': ['SmartX ProPhone']},
    {'category': '相机和摄像机', 'products': ['FotoSnap DSLR Camera']},
    {'category': '电视和家庭影院系统', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}
]


<span style="color:#00ff00">
gpt-4o-mini基本上能生成正确的输出。
</span>

它看起来像是输出了正确的数据，但它也输出了一堆文本，这些是多余的。这使得将其解析为 Python 字典列表更加困难。

## 四、更难的测试用例

找出一些在实际使用中，模型表现不如预期的查询。

In [17]:
customer_msg_4 = f"""
告诉我关于CineView电视的信息，那款8K的，还有Gamesphere游戏机，X款的。
我预算有限，你们有哪些电脑？"""

products_by_category_4 = find_category_and_product_v1(customer_msg_4,products_and_category)
print(products_by_category_4)

[
    {'category': '电视和家庭影院系统', 'products': ['CineView 8K TV']},
    {'category': '游戏机和配件', 'products': ['GameSphere X']},
    {'category': '电脑和笔记本', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}
]


In [19]:
customer_msg_5 = f"""
介绍一下CineView 8K TV"""

products_by_category_5 = find_category_and_product_v1(customer_msg_5,products_and_category)
print(products_by_category_5)

[]



<span style="color:#00ff00">
gpt-4o-mini对于这类用户输入，无法生成正确的输出。
</span>

## 五、修改指令以处理难测试用例

我们在提示中添加了以下内容，不要输出任何不在 JSON 格式中的附加文本，并添加了第二个示例，使用用户和助手消息进行 few-shot 提示。

<span style="color:#00ff00">
使用Claude 3.7 Sonnet 推荐的prompt优化方案，可以生成正确的输出。
</span>

In [38]:
def find_category_and_product_v2(user_input,products_and_category):
   """
   从用户输入中获取到产品和类别

   添加：不要输出任何不符合 JSON 格式的额外文本。
   添加了针对单个产品查询的特殊规则与示例。

   参数：
   user_input：用户的查询
   products_and_category：产品类型和对应产品的字典    
   """
   delimiter = "####"
   system_message = f"""
   您将提供客户服务查询。\
   客户服务查询将用{delimiter}字符分隔。
   仅输出一个 Python 对象列表，其中每个对象具有以下格式：
      'category': <必须是在下面的允许产品列表中找到的类别>;
      'products': <必须是在下面的允许产品列表中找到的产品>;

   特别注意以下几点:
   1. 如果客户询问特定产品（如"介绍CineView 8K TV"或"tell me about SmartX ProPhone"）：
      - 识别这是一个具体产品查询
      - 找出该产品所属的类别
      - 在响应中只包含该特定产品
      
   2. 如果客户询问整个类别（如"你们有哪些电视"或"what computers do you have"）：
      - 返回该类别的所有产品
   
   3. 格式要求：
      - 仅输出JSON格式的Python列表，不包含任何额外解释文本
      - 不要在JSON之外添加任何评论或说明
      - 确保输出可以被json.loads()函数解析

   类别和产品必须在客户输入中找到。
   如果客户输入提及了类别，则将其与允许产品列表中的正确产品相关联。
   如果客户输入提及了产品，则将其与允许类别中的正确类别相关联。
   如果未找到符合要求的产品或类别，则输出空列表。

   根据产品名称和产品类别与客户服务查询的相关性，列出所有相关的产品。
   
   允许的产品以 JSON 格式提供。
   每个项目的键代表类别。
   每个项目的值是该类别中的产品列表。
   允许的产品：{products_and_category}
   
   """
      
   # 针对介绍类查询的示例
   few_shot_user_1 = """介绍一下CineView 8K TV"""
   few_shot_assistant_1 = """[{'category': '电视和家庭影院系统', 'products': ['CineView 8K TV']}]"""
   
   # 针对"想要"类查询的示例
   few_shot_user_2 = """我想要最贵的电脑。你推荐哪款？"""
   few_shot_assistant_2 = """[{'category': '电脑和笔记本', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]"""
   
   messages = [  
      {'role': 'system', 'content': system_message},    
      {'role': 'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
      {'role': 'assistant', 'content': few_shot_assistant_1},
      {'role': 'user', 'content': f"{delimiter}{few_shot_user_2}{delimiter}"},  
      {'role': 'assistant', 'content': few_shot_assistant_2},
      {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
   ]

   return get_completion_from_messages(messages)

## 六、在难测试用例上评估修改后的指令

In [31]:
customer_msg_3 = f"""
告诉我关于smartx pro手机和fotosnap相机的信息，那款DSLR的。
另外，你们有哪些电视？"""

products_by_category_3 = find_category_and_product_v2(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

[{'category': '智能手机和配件', 'products': ['SmartX ProPhone']}, {'category': '相机和摄像机', 'products': ['FotoSnap DSLR Camera']}, {'category': '电视和家庭影院系统', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]


In [32]:
customer_msg_5 = f"""
介绍一下CineView 8K TV"""

products_by_category_5 = find_category_and_product_v2(customer_msg_5,products_and_category)
print(products_by_category_5)

[{'category': '电视和家庭影院系统', 'products': ['CineView 8K TV']}]


In [33]:
customer_msg_6 = f"""
介绍一下BlueWave Gaming Laptop"""

products_by_category_6 = find_category_and_product_v2(customer_msg_6,products_and_category)
print(products_by_category_6)

[{'category': '电脑和笔记本', 'products': ['BlueWave Gaming Laptop']}]


## 七、回归测试：验证模型在以前的测试用例上仍然有效

检查并修复模型以提高难以测试的用例效果，同时<span style="color:red">确保此修正不会对先前的测试用例性能造成负面影响</span>。

<span style="color:#00ff00">
此时模型生成输出与之前并不完全一致，但是因为用户提出预算有限，所以模型给出的列表中去掉了最贵的8k电视（虽然我们没有显式给模型每款电视具体的价格）。这是可以接受的。
</span>

In [35]:
customer_msg_0 = f"""如果我预算有限，我可以买哪款电视？"""

products_by_category_0 = find_category_and_product_v2(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

[{'category': '电视和家庭影院系统', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'SoundMax Soundbar', 'CineView OLED TV']}]


<span style="color:#00ff00">
而当用户不再提出预算有限时，模型给出的列表将包含8k电视。
</span>

In [36]:
customer_msg_0 = f"""我可以买哪款电视？"""

products_by_category_0 = find_category_and_product_v2(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

[{'category': '电视和家庭影院系统', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]


## 八、收集开发集进行自动化测试

当您要调整的开发集不仅仅是一小部分示例时，开始<span style="color:red">自动化测试过程</span>就变得有用了。

In [45]:
msg_ideal_pairs_set = [
    
    # eg 0
    {'customer_msg':"""如果我预算有限，买哪款电视比较好？""",
     'ideal_answer':{
        '电视和家庭影院系统': set(
            ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']
        )}
    },

    # eg 1
    {'customer_msg':"""我需要给我的智能手机买一个充电器。""",
     'ideal_answer':{
        '智能手机和配件': set(
            ['MobiTech PowerCase', 'MobiTech Wireless Charger']
        )}
    },

    # eg 2
    {'customer_msg':"""你们有些什么电脑？""",
     'ideal_answer':{
           '电脑和笔记本': set(
               ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']
               )
                }
    },

    # eg 3
    {'customer_msg':"""告诉我关于SmartX Pro Phone和FotoSnap DSLR Camera的信息。另外，你们有什么电视？""",
     'ideal_answer':{
        '智能手机和配件': set(
            ['SmartX ProPhone']),
        '相机和摄像机': set(
            ['FotoSnap DSLR Camera']),
        '电视和家庭影院系统': set(
            ['CineView 4K TV', 'SoundMax Home Theater','CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV'])
        }
    }, 
    
    # eg 4
    {'customer_msg':"""告诉我关于CineView电视，8K的那款，Gamesphere游戏机，X款。另外，你们有什么电脑？""",
     'ideal_answer':{
        '电视和家庭影院系统': set(
            ['CineView 8K TV']),
        '游戏机和配件': set(
            ['GameSphere X']),
        '电脑和笔记本': set(
            ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'])
        }
    },
    
    # eg 5
    {'customer_msg':"""你们有些什么智能手机？""",
     'ideal_answer':{
           '智能手机和配件': set(
               ['SmartX ProPhone', 'MobiTech PowerCase', 'SmartX MiniPhone', 'MobiTech Wireless Charger', 'SmartX EarBuds']
               )
                    }
    },
    # eg 6
    {'customer_msg':"""我预算有限。你能推荐一些智能手机给我吗？""",
     'ideal_answer':{
        '智能手机和配件': set(
            ['SmartX EarBuds', 'SmartX MiniPhone', 'MobiTech PowerCase', 'SmartX ProPhone', 'MobiTech Wireless Charger']
        )}
    },

    # eg 7 # this will output a subset of the ideal answer
    {'customer_msg':"""对于我喜欢赛车游戏的朋友来说，哪些游戏机比较合适？""",
     'ideal_answer':{
        '游戏机和配件': set([
            'GameSphere X',
            'ProGamer Controller',
            'GameSphere Y',
            'ProGamer Racing Wheel',
            'GameSphere VR Headset'
     ])}
    },
    # eg 8
    {'customer_msg':"""给我喜欢拍视频的朋友挑选个什么礼物比较好？""",
     'ideal_answer': {
        '相机和摄像机': set([
        'FotoSnap DSLR Camera', 'ActionCam 4K', 'FotoSnap Mirrorless Camera', 'ZoomMaster Camcorder', 'FotoSnap Instant Camera'
        ])}
    },
    
    # eg 9
    {'customer_msg':"""我想要一个热水浴缸时间机器。""",
     'ideal_answer': []
    }
    
]


## 九、通过与理想答案比较来评估测试用例

In [40]:
import json
def eval_response_with_ideal(response,
                              ideal,
                              debug=False):
    """
    评估回复是否与理想答案匹配
    
    参数：
    response: 回复的内容
    ideal: 理想的答案
    debug: 是否打印调试信息
    """
    if debug:
        print("回复：")
        print(response)
    
    # json.loads() 只能解析双引号，因此此处将单引号替换为双引号
    json_like_str = response.replace("'",'"')
    
    # 解析为一系列的字典
    l_of_d = json.loads(json_like_str)
    
    # 当响应为空，即没有找到任何商品时
    if l_of_d == [] and ideal == []:
        return 1
    
    # 另外一种异常情况是，标准答案数量与回复答案数量不匹配
    elif l_of_d == [] or ideal == []:
        return 0
    
    # 统计正确答案数量
    correct = 0    
    
    if debug:
        print("l_of_d is")
        print(l_of_d)

    # 对每一个问答对  
    for d in l_of_d:

        # 获取产品和目录
        cat = d.get('category')
        prod_l = d.get('products')
        # 有获取到产品和目录
        if cat and prod_l:
            # convert list to set for comparison
            prod_set = set(prod_l)
            # get ideal set of products
            ideal_cat = ideal.get(cat)
            if ideal_cat:
                prod_set_ideal = set(ideal.get(cat))
            else:
                if debug:
                    print(f"没有在标准答案中找到目录 {cat}")
                    print(f"标准答案: {ideal}")
                continue
                
            if debug:
                print("产品集合：\n",prod_set)
                print()
                print("标准答案的产品集合：\n",prod_set_ideal)

            # 查找到的产品集合和标准的产品集合一致
            if prod_set == prod_set_ideal:
                if debug:
                    print("正确")
                correct +=1
            else:
                print("错误")
                print(f"产品集合: {prod_set}")
                print(f"标准的产品集合: {prod_set_ideal}")
                if prod_set <= prod_set_ideal:
                    print("回答是标准答案的一个子集")
                elif prod_set >= prod_set_ideal:
                    print("回答是标准答案的一个超集")

    # 计算正确答案数
    pc_correct = correct / len(l_of_d)
        
    return pc_correct

In [46]:
print(f'用户提问: {msg_ideal_pairs_set[7]["customer_msg"]}')
print(f'标准答案: {msg_ideal_pairs_set[7]["ideal_answer"]}')

用户提问: 对于我喜欢赛车游戏的朋友来说，哪些游戏机比较合适？
标准答案: {'游戏机和配件': {'ProGamer Racing Wheel', 'ProGamer Controller', 'GameSphere X', 'GameSphere Y', 'GameSphere VR Headset'}}


In [47]:
response = find_category_and_product_v2(msg_ideal_pairs_set[7]["customer_msg"],
                                         products_and_category)
print(f'回答: {response}')

eval_response_with_ideal(response,
                              msg_ideal_pairs_set[7]["ideal_answer"])

回答: [{'category': '游戏机和配件', 'products': ['GameSphere X', 'ProGamer Controller', 'GameSphere Y', 'ProGamer Racing Wheel', 'GameSphere VR Headset']}]


1.0

## 十、在所有测试用例上运行评估，并计算正确的用例比例

注意：如果任何 API 调用超时，将无法运行

In [49]:
import time

score_accum = 0
for i, pair in enumerate(msg_ideal_pairs_set):
    time.sleep(10)
    print(f"示例 {i}")
    
    customer_msg = pair['customer_msg']
    ideal = pair['ideal_answer']
    
    # print("Customer message",customer_msg)
    # print("ideal:",ideal)
    response = find_category_and_product_v2(customer_msg,
                                                      products_and_category)

    
    # print("products_by_category",products_by_category)
    score = eval_response_with_ideal(response,ideal,debug=False)
    print(f"{i}: {score}")
    score_accum += score
    

n_examples = len(msg_ideal_pairs_set)
fraction_correct = score_accum / n_examples
print(f"正确比例为 {n_examples}: {fraction_correct}")

示例 0
错误
产品集合: {'CineView OLED TV', 'SoundMax Home Theater', 'SoundMax Soundbar', 'CineView 4K TV'}
标准的产品集合: {'SoundMax Home Theater', 'CineView OLED TV', 'CineView 4K TV', 'SoundMax Soundbar', 'CineView 8K TV'}
回答是标准答案的一个子集
0: 0.0
示例 1
错误
产品集合: {'MobiTech Wireless Charger'}
标准的产品集合: {'MobiTech PowerCase', 'MobiTech Wireless Charger'}
回答是标准答案的一个子集
1: 0.0
示例 2
2: 1.0
示例 3
3: 1.0
示例 4
4: 1.0
示例 5
5: 1.0
示例 6
6: 1.0
示例 7
7: 1.0
示例 8
8: 1.0
示例 9
9: 1
正确比例为 10: 0.8


使用 Prompt 构建应用程序的工作流程与使用监督学习构建应用程序的工作流程<span style="color:red">非常不同</span>。

因此，我们认为这是需要记住的一件好事，当您正在构建监督学习模型时，会感觉到迭代速度快了很多。

如果您并未亲身体验，可能会惊叹于仅有手动构建的极少样本，就可以产生高效的评估方法。您可能会认为，仅有 10 个样本是不具备统计意义的。但当您真正运用这种方式时，您可能会对<span style="color:red">向开发集中添加一些复杂样本</span>所带来的效果提升感到惊讶。

这对于帮助您和您的团队找到有效的 Prompt 和有效的系统非常有帮助。

在本章中，输出可以被定量评估，就像有一个期望的输出一样，您可以判断它是否给出了这个期望的输出。

在下一章中，我们将探讨如何在更加模糊的情况下评估我们的输出。即正确答案可能不那么明确的情况。